# Modelos de prediccion
Vamos a probar distintos enfoques:
Enfoques de serie de tiempo:
* Suavizado exponencial #investigar

Enfoques Machine Learning:
* XGboost: Emiii
* LIGHT_GBM: Rugo
* Catboost: Vano

In [3]:
from DataLoader import importar_ventas,importar_cluster_stores,leer_csv,agregar_fourier,preparar_test

from DataLoader import generar_rolling_features,crear_archivo_kaggle
import xgboost as xgb
import pandas as pd

In [4]:
ventas=importar_ventas()

In [3]:
ventas["demand"].describe()

count    5.206793e+06
mean     4.936933e+00
std      3.571045e+00
min      1.000000e+00
25%      3.000000e+00
50%      4.000000e+00
75%      6.000000e+00
max      1.150000e+02
Name: demand, dtype: float64

In [5]:
test=preparar_test("ids_test.csv")
#test

In [6]:
agregar_fourier(ventas,2)
agregar_fourier(test,2)


In [7]:
ventas=generar_rolling_features(ventas,"mean_price",[1,2,3,4],["subgroup_cod","store_cod"])
ventas=generar_rolling_features(ventas,"demand",[1,2,3,4],["subgroup_cod","store_cod"])

KeyboardInterrupt: 

In [7]:
#busquemos otra fecha para probar esto objetivo 01-01-2023 a 07-01-2023

# ventas_2023=ventas[ventas["date"]<"02-01-2023"]
# test_2023=ventas_2023.groupby(["subgroup_cod","store_cod"]).tail(7)
# train_2023=ventas_2023.drop(test_2023.index)

ventas.columns

Index(['date', 'demand', 'mean_price', 'std_price', 'max_price', 'min_price',
       'cluster', 'store_cod', 'subgroup_cod', 'demand_lag_1', 'demand_lag_2',
       'demand_lag_3', 'demand_lag_4', 'demand_lag_5', 'demand_lag_6',
       'demand_lag_7', 'day', 'month', 'year', 'anual_sin_1', 'anual_cos_1',
       'mensual_sin_1', 'mensual_cos_1', 'anual_sin_2', 'anual_cos_2',
       'mensual_sin_2', 'mensual_cos_2', 'mean_price_rolling_mean_7d',
       'mean_price_rolling_sum_7d', 'mean_price_rolling_mean_14d',
       'mean_price_rolling_sum_14d', 'mean_price_rolling_mean_21d',
       'mean_price_rolling_sum_21d', 'mean_price_rolling_mean_28d',
       'mean_price_rolling_sum_28d', 'demand_rolling_mean_7d',
       'demand_rolling_sum_7d', 'demand_rolling_mean_14d',
       'demand_rolling_sum_14d', 'demand_rolling_mean_21d',
       'demand_rolling_sum_21d', 'demand_rolling_mean_28d',
       'demand_rolling_sum_28d'],
      dtype='object')

In [41]:
import numpy as np
FEATURES=[
   #'mean_price', 
    'cluster', 
    #'store_cod', 
    'subgroup_cod',
    #'demand_lag_1', #'demand_lag_2', 'demand_lag_3', 
    #'demand_lag_4','demand_lag_5', 'demand_lag_6', 'demand_lag_7', 'day', 'month', 
    'year',
   'anual_sin_1', 
    'anual_cos_1',
   'mensual_sin_1', 
    'mensual_cos_1',
   #'anual_sin_2', 
    #'anual_cos_2',
    'mensual_sin_2', 
    #'mensual_cos_2',
   # "std_price",
    "max_price", 
   "min_price",
    #'mean_price_rolling_mean_7d',
   #    'mean_price_rolling_sum_7d',# 'mean_price_rolling_mean_14d',
       #'mean_price_rolling_sum_14d', 
   # 'mean_price_rolling_mean_21d',
     #  'mean_price_rolling_sum_21d', 'mean_price_rolling_mean_28d',
    #  'mean_price_rolling_sum_28d', 
    #'demand_rolling_mean_7d',
    #  'demand_rolling_sum_7d', 
    'demand_rolling_mean_14d',
     #  'demand_rolling_sum_14d', #'demand_rolling_mean_21d',
       #'demand_rolling_sum_21d', #'demand_rolling_mean_28d',
      # 'demand_rolling_sum_28d'
] #recordar que ya probamos con fourier 3 y 4 y fue un fracaso

TARGET=['demand']

test_interno=ventas.groupby(["subgroup_cod","store_cod"]).tail(7)
train=ventas
train_interno=ventas.drop(test_interno.index)

X_train=train[FEATURES]
y_train=train[TARGET]
X_test_real=test[FEATURES]

X_train_interno=train_interno[FEATURES]
y_train_interno=train_interno[TARGET]

X_test_interno=test_interno[FEATURES]
y_test_interno=test_interno[TARGET]

y_train_interno_log=np.log1p(y_train_interno)
y_test_interno_log=np.log1p(y_test_interno)


# X_test_2023=test_2023[FEATURES]
# y_test_2023=test_2023[TARGET]

# X_train_2023=train_2023[FEATURES]
# y_train_2023=train_2023[TARGET]

## Modelo interno

In [45]:
from sklearn.metrics import r2_score

reg = xgb.XGBRegressor(
    n_estimators=500,
    eval_metric="rmse",
    eta=0.1,
    max_depth=32,
   #colsample_bytree=0.9, #reducir el numero de columnas aumenta la importancia de std_price
    early_stopping_rounds=50 
)

reg.fit(
    X_train_interno, y_train_interno_log
   ,eval_set=[(X_test_interno, y_test_interno_log)]   
   ,verbose=50                       
)
#sin anual_sin_2 450 estimadores =0.11283 y con da lo mismo

[0]	validation_0-rmse:0.47730


KeyboardInterrupt: 

In [40]:
pd.DataFrame(reg.feature_importances_,columns=["importance"],index=reg.feature_names_in_).sort_values(by="importance",ascending=False)

,importance
max_price,0.263704
min_price,0.209086
demand_rolling_mean_14d,0.083938
year,0.069071
mensual_cos_1,0.066970
demand_rolling_sum_7d,0.059225
demand_rolling_sum_14d,0.055347
anual_sin_1,0.041263
anual_cos_1,0.027554
cluster,0.027519


## Modelo kraggel


In [8]:
from sklearn.metrics import mean_squared_error
import numpy as np

prediccion=reg.predict(X_test_interno)


# y_true_log y y_pred_log están en escala log1p
y_true_original = np.expm1(y_test_interno_log)
y_pred_original = np.expm1(prediccion)

rmse_original = np.sqrt(mean_squared_error(y_true_original, y_pred_original))

print(rmse_original)
print(r2_score(y_true_original,y_pred_original))
#orden_bueno = pd.DataFrame({
#    "store_subgroup_date_id": test["store_subgroup_date_id"],
#    "prediccion": prediccion.ravel()
#})

#kaggle=crear_archivo_kaggle(orden_bueno)

NameError: name 'reg' is not defined

In [11]:
#kaggle.describe()
400*48/60/60

5.333333333333333

# Probemos hacer un modelo por categoria

In [1]:
from DataLoader import importar_ventas,leer_csv,agregar_fourier,preparar_test
from DataLoader import generar_rolling_features,crear_archivo_kaggle
import numpy as np
import xgboost as xgb
import pandas as pd

In [2]:
ventas=importar_ventas()
ventas_computing=ventas[ventas["category"]=="Electronics"]

In [5]:
agregar_fourier(ventas_computing,2)

C:\Users\emili\OneDrive\Desktop\Economy\DataLoader.py:211: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["date"]=pd.to_datetime(df["date"])
C:\Users\emili\OneDrive\Desktop\Economy\DataLoader.py:217: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"anual_sin_{i}"]=sin(2*i*pi*dias/periodo_anual)
C:\Users\emili\OneDrive\Desktop\Economy\DataLoader.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [48]:
ventas_computing=generar_rolling_features(ventas_computing,"demand",[1,4],["subgroup_cod","store_cod"])

In [49]:
ventas_computing.columns

Index(['date', 'demand', 'mean_price', 'std_price', 'max_price', 'min_price',
       'group', 'category', 'cluster', 'store_cod', 'subgroup_cod', 'day',
       'month', 'year', 'anual_sin_1', 'anual_cos_1', 'mensual_sin_1',
       'mensual_cos_1', 'anual_sin_2', 'anual_cos_2', 'mensual_sin_2',
       'mensual_cos_2', 'mean_price_rolling_mean_7d',
       'mean_price_rolling_sum_7d', 'mean_price_rolling_mean_28d',
       'mean_price_rolling_sum_28d', 'demand_rolling_mean_7d',
       'demand_rolling_sum_7d', 'demand_rolling_mean_28d',
       'demand_rolling_sum_28d'],
      dtype='object')

In [2]:
TARGET=["demand"]
FEATURES=[#'mean_price', #'std_price', 
          'max_price', 'min_price',
        'cluster', #'store_cod', 
        # 'subgroup_cod', #'day',
       #'month', 
        'year', 'anual_sin_1',
    'anual_cos_1', 'mensual_sin_1',
       #'mensual_cos_1', 
        #'anual_sin_2', 'anual_cos_2', 
    'mensual_sin_2', #'demand_rolling_mean_7d',
       'demand_rolling_sum_7d', 
    #'demand_rolling_mean_28d',
       #'demand_rolling_sum_28d'
       #'mensual_cos_2',
        # 'mean_price_rolling_mean_7d',
       #'mean_price_rolling_sum_7d', 
    #'mean_price_rolling_mean_28d',
       #'mean_price_rolling_sum_28d'
          
         ]
test_interno=ventas_computing.groupby(["subgroup_cod","store_cod"]).tail(7)
train_interno=ventas_computing.drop(test_interno.index)

X_train_interno=train_interno[FEATURES]
y_train_interno=train_interno[TARGET]

X_test_interno=test_interno[FEATURES]
y_test_interno=test_interno[TARGET]




NameError: name 'ventas' is not defined

In [140]:

reg = xgb.XGBRegressor(
    n_estimators=500,
    eval_metric="rmse",
    objective="reg:squaredlogerror",
    eta=0.05,
    max_depth=10,
   # alpha=300,
   colsample_bytree=1, #reducir el numero de columnas aumenta la importancia de std_price
    early_stopping_rounds=50
)

reg.fit(
    X_train_interno, y_train_interno
   ,eval_set=[(X_test_interno, y_test_interno)]   
   ,verbose=50                       
) #ponele que bajamos algo 

[0]	validation_0-rmse:2.85330
[50]	validation_0-rmse:2.19163
[100]	validation_0-rmse:2.13929
[150]	validation_0-rmse:2.13285
[200]	validation_0-rmse:2.12891
[250]	validation_0-rmse:2.12788
[300]	validation_0-rmse:2.12800
[325]	validation_0-rmse:2.12855


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             device=None, early_stopping_rounds=50, enable_categorical=False,
             eta=0.05, eval_metric='rmse', feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=500,
             n_jobs=None, ...)

In [141]:
pd.DataFrame(reg.feature_importances_,columns=["importance"],index=reg.feature_names_in_).sort_values(by="importance",ascending=False)

,importance
max_price,0.267903
mensual_sin_2,0.186416
min_price,0.175157
year,0.126397
demand_rolling_sum_7d,0.082164
anual_sin_1,0.052880
cluster,0.042565
anual_cos_1,0.036059
mensual_sin_1,0.030460


In [142]:
from sklearn.metrics import r2_score
prediccion=reg.predict(X_test_interno)
r2_score(y_test_interno,prediccion)

0.44058603048324585

In [12]:
from DataLoader import leer_csv
transacciones=leer_csv("eci_transactions")